<img src="../static/logopython.png" alt="Logo Python" style="width: 400px; display: inline"/>
<img src="../static/deimoslogo.png" alt="Logo Deimos" style="width: 400px; display: inline"/>

# Clase 2a: Herramientas para programar como un pythonista

Ya hemos visto que la orientación a objetos es solo uno de los paradigmas que podemos usar cuando programamos en Python. Al margen de frameworks, como <a href="https://www.djangoproject.com/">Django</a>, orientado al desarrollo web, existen una serie de estructuras y herramientas en el propio lenguaje que nos ayudan a tomar decisiones cuando creamos una aplicación Python. Veremos a continuación las más populares

## Módulos y namespaces

Como ya dijimos en el capítulo anterior, Python proporciona algunas herramientas que ayudan a la organización de nuestro código, más allá del paradigma de la programación orientada a objetos. Los módulos y los namespaces son posiblemente los más conocidos. Veámoslos a continuación

### Namespaces

Como ya vimos en la introducción, un *nombre* en Python es lo que en otros lenguajes de programación llamaríamos una *variable*

In [6]:
a = 12
b = 'B'
c = [1, 2, 3, 4]
print(a, b, c)

12 B [1, 2, 3, 4]


Pero un *nombre* también puede almacenar una función

In [5]:
def func():
    print('Esto es una función')
 
f = func
f()

Esto es una función


Un *namespace* no es otra cosa que un espacio en memoria donde se almacenan un conjunto de nombres. Se crean de manera automática, sin que nosotros hagamos nada.

### Modules

Los módulos van unidos conceptualmente a los *namespaces*. Un módulo es un fichero que contiene código Python, ya sea en forma de clases, funciones o simplemente una lista de nombres. Y cada módulo obtiene su propio *namespace*. Por eso no podemos tener en el mismo fichero dos clases o funciones con el mismo nombre

Por otro lado, <strong>cada *namespace* está totalmente aislado del resto</strong>, de manera que dos módulos pueden tener clases o funciones con el mismo nombre.

Por último, cuando ejecutamos un script Python, el intérprete crea un módulo llamado *\__main\__*, con su propio namespace

<div class="alert alert-info">Los módulos constituyen las capas de abstracción más naturales en el lengujar Python. Permiten separar el código en varias partes que comparten funcionalidad</div>

Para poder usar módulos en nuestra aplicación, tenemos que importarlos. Veremos a continuación cómo hacerlo

#### import modulename

Mediante esta instrucción, obtenemos acceso a todo el espacio de nombres de un módulo. Deberemos preceder cada elemento del 
módulo con el nombre del módulo.

In [12]:
# Importamos el módulo math y podemos acceder a sus nombres. Es seguro porque todos los nombres del módulo requieren que
# pongamos delante el nombre del módulo en si.
import math
print(math.pi)

3.141592653589793


#### from module import name

Podemos también importar un nombre completo de un módulo. No será necesario preceder el nombre de cada nombre con el nombre del módulo, de manera que puede chocar con nombres que tengamos en nuestro namespace

In [13]:
# Importamos solo el nombre pi. Considerado mala práctica, porque los nombres importados podrían chocar con los nuestros
from math import pi
print(pi)

3.141592653589793


#### from module import *

También podemos importar todos los nombres de un módulo.

In [16]:
# Importamos todos los nombres de un módulo. Considerado mala práctica, por lo mismo que en el caso anterior
from math import *
print(pi)
print(log2(16))

3.141592653589793
4.0


<div class="alert alert-info">Cuando importamos un módulo, Python lo busca en el directorio actual. Si no lo encuentra, lo buscará en su *path*, que está definido en tiempo de compilación</div>

## Paquetes

Igual que un fichero de código Python se considera un módulo, <strong>un directorio que contiene varios ficheros se considera un paquete ,siempre y cuando uno de esos ficheros se llame *\__init\__.py*</strong>. En este fichero se suelen almacenar nombres que serán usados por el resto de módulos del mismo, si son necesarios. Pero es habitual, e incluso buena práctica, que el fichero esté vacío.

<div class="alert alert-info">Los paquetes son una capa de abstracción superior a los módulos. Cada paquete suele contener módulos con funcionalidades complementarias</div>

Para importar un módulo (fichero) dentro de un paquete (directorio), escribimos: *import package.module*. Python buscará el fichero *module.py* dentro del directorio *package*. Primero en el directorio actual, y si no lo encuentra, acudirá a su propio *path* a buscarlo.

<div class="alert alert-success"><strong>Buenas prácticas</strong>: Se considera buena práctica definir *alias* al cargar paquetes con una estructura de directorios demasiado profunda, para evitar escribir de más al acceder a los nombres del módulo importado. Ejemplo: *import very.deep.module as mod*</div>

Ahora que ya sabemos cómo encapsular nuestro código, tanto si usamos programación orientada a objetos como si no, veamos ciertas herramientas que nos ayudan a diseñar programas de manera más eficiente y elegante. Empezaremos con los *context managers*

## Context Managers

Los *context managers* son objetos especiales que proporcionan información contextual adicional a una acción. Se aseguran de que sucedan cosas tanto **antes** como **después** de que la acción se lleve a cabo. Para especificar que un bloque de código se ejecuta bajo la supervisión de un *context manager*, usamos la palabra clave reservada *with*

El ejemplo clásico es leer o escribir en un fichero. Mediante un *context manager*:

<ul>
    <li>Me aseguro de que el fichero se abra antes de comenzar la operación de lectura/escritura</li>
    <li>Me aseguro de que el fichero se cierre después de terminar la operación de lectura/escritura</li>
</ul>

El ejemplo básico sería así

In [17]:
# Abrimos un fichero y escribimos texto dentro. No es necesario que lo cerremos. Se encarga el context manager
with open('file.txt', 'w') as f:
    f.write('hola')

Podríamos implementar nuestra propia versión de este *context manager*

In [19]:
class CustomOpen(object):
    # Constructor de la clase
    def __init__(self, filename):
      self.file = open(filename)

    # Esto es lo que se devuelve en el with, y va a la variable f
    def __enter__(self):
        return self.file

    # Esto se ejecuta antes de terminar el context manager
    def __exit__(self, ctx_type, ctx_value, ctx_traceback):
        self.file.close()

with CustomOpen('file.txt') as f:
    contents = f.read()
    print(contents)

hola


Python proporciona un *decorator* especial para facilitar la creación de *context manager*. Se denomina *contextmanager* y se encuentra dentro del paquete *contextlib*, presente en la instalación estándar.

In [22]:
# Implementación del mismo context manager usando contextlib.contextmanager
from contextlib import contextmanager

@contextmanager
def custom_open(filename):
    f = open(filename)
    try:
        # La función se ejecuta hasta que encuentra una llamada a yield, y eso es lo que se devuelve en f
        yield f
        
    # Esto es lo último que se ejecuta en el context manager
    finally:
        f.close()

with custom_open('file.txt') as f:
    contents = f.read()
    print(contents)

hola


<div class="alert alert-info">No te preocupes del significado de *yield* ahora. Lo veremos cuando estudiemos los generadores. Quedémonos con que devuelve un valor</div>

<div class="alert alert-success"><strong>Buenas prácticas</strong>: Usa el zen de Python para decidir qué método de los dos anteriores utilizar. Implementa tu propia clase *Context Manager* si el código a encapsular contiene mucha lógica. Usa el módulo *contextmanager* del paquete *contextlib* si lo que quieres encapsular es una acción relativamente sencilla</div>

## Iterables

Los iterables son elementos que se pueden recorrer. Como las listas, los diccionarios o los *strings*. 

In [3]:
# Una lista es un iterable
a_list = [1, 2, 3]
for a in a_list:
    # Tip: para que print no meta un salto de línea al final de cada llamada, pasarle un segundo argumento end=' '
    print (a, end=' ')
    
# Un string también es un iterable
a_string = "hola"
for a in a_string:
    print(a, end=' ')

1 2 3 h o l a 

También es posible crear un *iterable* a partir de una *comprehension list* (veremos *comprehension* al final de este tema)

In [94]:
# Construimos iterable (lista) a partir de una comprehension list
numeros = [x*x for x in range(3)]
for n in numeros:
    print(n, end=' ')

0 1 4 

Otro ejemplo de *iterable* a partir de una expresión de tipo *comprehension list*: los 15 primeros dígitos de la <a href="https://es.wikipedia.org/wiki/Sucesi%C3%B3n_de_Fibonacci">Sucesión de Fibonnaci</a>

In [95]:
# 15 primeros dígitos de la secuencia de Fibonacci con comprehension list
fib = [0,1]
[fib.append(fib[-1] +fib[-2]) for ctr in range(15)]
print(fib)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]


También podemos demostrar lo *nerds* que somos usando la <a href="http://stackoverflow.com/a/6504560">Fórmula de Binet</a> para generar la lista

In [63]:
fib = [round(0.4472135954999579392818347337462552470881236719223051448541*(pow(1.6180339887498948482045868343656381177203091798057628621354,n) - pow(-0.618033988749894848204586834365638117720309179805762862135,n))) for n in range(17)]
print(fib)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]


Incluso podríamos implementarlo mediante <a href="https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions">expresiones lambda</a> y <a href="https://docs.python.org/3/library/functools.html#functools.reduce">reduce</a>, frecuentes en programación funcional. Aunque Guido Van Rosuum prefiere las *list comprehension*

In [65]:
import functools

fib_function = lambda n:functools.reduce(lambda x,n:[x[1],x[0]+x[1]], range(n),[0,1])[0]

fib = [fib_function(n) for n in range(17)]
print(fib)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]


Al trabajar con iterables, en cualquier caso, es una buena idea usar el módulo *itertools*

<div class="alert alert-success"><strong>Buenas prácticas</strong>: Usa el módulo <a href="https://docs.python.org/3.4/library/itertools.html">*itertools*</a> siempre que trabajes con iterables. Tiene muchas utilidades que te ahorrarán trabajo.</div>

Veamos a continuación algunos ejemplos de cosas que podemos hacer con *itertools*

In [6]:
# Genera todas las permutaciones de 4 elementos usando itertools
import itertools

elementos = [1, 2, 3, 4]
permutaciones = itertools.permutations(elementos)
for p in permutaciones:
    print(p, end=' - ')

(1, 2, 3, 4) - (1, 2, 4, 3) - (1, 3, 2, 4) - (1, 3, 4, 2) - (1, 4, 2, 3) - (1, 4, 3, 2) - (2, 1, 3, 4) - (2, 1, 4, 3) - (2, 3, 1, 4) - (2, 3, 4, 1) - (2, 4, 1, 3) - (2, 4, 3, 1) - (3, 1, 2, 4) - (3, 1, 4, 2) - (3, 2, 1, 4) - (3, 2, 4, 1) - (3, 4, 1, 2) - (3, 4, 2, 1) - (4, 1, 2, 3) - (4, 1, 3, 2) - (4, 2, 1, 3) - (4, 2, 3, 1) - (4, 3, 1, 2) - (4, 3, 2, 1) - 

¿Qué tal si calculamos el producto cartesiano de dos iterables? De hecho, no tienen porqué ser listas. Puede ser cualquier iterable, como un string

In [7]:
# Genera el producto cartesiano de dos iterables definidos como string
import itertools

cartesiano = itertools.product('ABC', '123')
for c in cartesiano:
    print(c, end=' - ')

('A', '1') - ('A', '2') - ('A', '3') - ('B', '1') - ('B', '2') - ('B', '3') - ('C', '1') - ('C', '2') - ('C', '3') - 

O las combinaciones de elementos de un iterable, cogidas de 2 en 2

In [8]:
# Genera combinaciones de elementos de un iterable, cogidas de 2 en 2
import itertools
bases = "ATCG"

comb = itertools.combinations(bases, 2)
for c in comb:
    print(c, end=' - ')

('A', 'T') - ('A', 'C') - ('A', 'G') - ('T', 'C') - ('T', 'G') - ('C', 'G') - 

¿Y si agrupamos listas de elementos en función de un criterio, como la longitud de los mismos?

In [27]:
# En este ejercicio agruparemos nombres de personas por su longitud, usando itertools
import itertools

# Context manager en acción...
with open('nombres.txt') as f:
    names = list(f)
    
    # Necesitamos eliminar el salto de línea al final de cada nombre leído del fichero
    # Esto es una list comprehension. Las veremos al final de este tema
    names = [name.rstrip() for name in names]

    # Esto es por si no estuviera el fichero
    #names = ['Paco', 'Julia', 'Javier', 'Sara', 'Jose','Gala', 'Jorge', 'Marta', 'Alex', 'Carmen']

    # Groupby genera, a partir de un iterable, iterables con elementos que devuelvan el mismo resultado al aplicarles una función
    # El problema es que el iterable se detiene cuando un elemento ya no devuelve el mismo resultado que el anterior, sin importar si
    # ya ha generado iterables que devuelvan ese mismo resultado.
    # En nuestro caso, si la lista de nombres no estuviera ordenada por longitud, crearía un iterable para 'Paco' (longitud 4),
    # otro iterable para 'Julia' y 'Javier' (longitud 5), luego otro para 'Sara' (longitud 4), etc. Lo interesante sería que
    # 'Paco' y 'Sara' fueran en el mismo iterable, pues tienen la misma longitud.
    # Para conseguir eso, ordenamos la lista mediante el mismo criterio que usaremos en la generación de iterables.
    # Ver lo que pasa si llamamos a groupby sobre 'names' a secas
    grupos = itertools.groupby(sorted(names, key=len), len)

    for n, it in grupos:
        print("Nombres con {0} letras:".format(n), end=' ')
        for name in it:
            print(name, end=' ')
        print()

Nombres con 4 letras: Paco Sara Jose Gala Alex 
Nombres con 5 letras: Julia Jorge Marta 
Nombres con 6 letras: Javier Carmen 


## Iteradores

Por ahora hemos visto lo que es un iterable: un elemento que se puede recorrer. Pero vamos a precisar un poco más la definición exacta, para poder introducir un nuevo tipo de datos, clave en Python: <strong>el iterador</strong>

<div class="alert alert-info">Un iterable es un elemento capaz de devolver un iterador, con la finalidad de poder iterar sobre todos los elementos que contiene. Cualquier clase puede convertirse en iterable implementando un método *\__iter\__()* que devuelva un objeto de tipo *iterator*</div>

Muy bien, ya sabemos qué es un iterable, en terminología de Python. Ahora, definamos qué es un objeto de tipo *iterator*

<div class="alert alert-info">Un iterador es una clase que contiene un método *\__next\__()*, que devuelve el siguiente elemento de una secuencia (un *iterable*). Eso implica que, para construir un iterador, necesitamos un objeto *iterable*.</div>

Veamos más claro el concepto con este ejemplo

In [41]:
# Una lista es un iterable
a_list = [1, 2, 3]

# Creamos dos iteradores a partir de la lista, mediante la función iter()
it1 = iter(a_list)
it2 = iter(a_list)

# Vamos recorriendo el iterador 1, consumiendo un elemento en cada paso, mediante la función next()
print(next(it1))
print(next(it1))

# Ahora recorremos el iterador 2, que seguirá en la primera posición
print(next(it2))

1
2
1


En el ejemplo anterior, hemos partido de un tipo nativo de Python: una lista. Este tipo nativo consiste, internamente, en una clase con un método *\__iter\__()*. Mediante la llamada a la función iter(), hemos llamado realmente a *a_list.\__iter\__()*, que nos ha devuelto un iterador. De esta forma, hemos construido dos iteradores: *it1* e *it2*.

Posteriormente, hemos llamado a la función *next()* pasándole el iterador como argumento. Eso hace que se llame al método *it1.\__next\__()* (lo mismo con *it2*). Y cada llamada a *next()*, nos devuelve el siguiente elemento de la lista.

Para entender el funcionamiento interno de un iterador, podemos construirnos uno sencillo: <strong>Iterador que construye la secuencia de Fibonacci</strong>

In [90]:
# Construir iterador para generar secuencia de Fibonacci
import itertools

class Fib:
    '''Iterador que genera la secuencia de Fibonacci'''
    
    # Constructor
    def __init__(self):
        self.prev = 0
        self.curr = 1

    # Esto es llamado cada vez que se llama a iter(Fib). Algo que for hace automáticamente, pero lo podríamos hacer a mano.
    # La función __iter__ debe devolver cualquier objeto que implemente __next__. En este caso, la propia clase
    def __iter__(self):
        return self

    # Esto es llamado cada vez que se llama a next(Fib). Algo que for hace automáticamente, pero lo podríamos hacer a mano.
    def __next__(self):
        value = self.curr
        self.curr += self.prev
        self.prev = value
        return value
   
# f guarda un iterador. En este momento no se ha llamado a nada, solo se ha construido el iterador
f = Fib()

# Recorremos nuestro iterador, llamando a next(). Dentro del for se llama automáticamente a iter(f)
print(0, end=' ')
for n in range(16):
    print(next(f), end=' ')
    
# Otra manera más elegante de hacerlo, con itertools
#print([0] + list(itertools.islice(f, 0, 16)))

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 

Es importante destacar que, la clase *Fib*, <strong>es a la vez un *iterator* y un *iterable*</strong>

<ul>
    <li>Un *iterable* porque contiene un método *\__iter()\__*. Este método ha de devolver un iterador (una clase con un método *\__next\__()*), de manera que se devuelve a si misma.</li>
    <li>Un *iterator* porque contiene un método *\__next()\__*. El siguiente elemento de la secuencia a devolver se va calculando mediante una fórmula matemática, pero podría estar guardado en algún lado (en una base de datos, en una caché...), si quisiéramos evitar realizar el cálculo, por ser excesivamente complejo</li>
</ul>

La clave que hace al iterador una herramienta tan poderosa es que <strong>los valores que se devuelven no están almacenados en memoria, sino que se producen *on the fly*</strong>.

Cuando recorremos un generador hasta el final, se dice que lo *agotamos* (en inglés: *we exhaust the generator*). Para poder volver a recorrerlo, hay que construirlo de nuevo. En nuestro ejemplo, el iterador no tiene fin, y sigue generando valores hasta el infinito (en la práctica: hasta que sobrepasemos el tamaño de un dato numérico en Python).

<div class="alert alert-info">Un iterador es una *lazy factory*. Solo genera valores cuando se le piden. De manera que, aunque lo usemos para generar una secuencia potencialmente infinita de elementos, solo tendremos en memoria cada vez el elemento de la secuencia con el que estemos trabajando. Nuestro iterador tiene un orden de crecimiento O(1), ¡y eso es una pasada!</div>

Por último destacar que, si queremos empezar a recorrer un *iterator* desde la primera posición una segunda vez, tendremos que <strong>instanciarlo de nuevo</strong>

Los iteradores son un <strong>concepto fundamental de Python</strong>. Son la base para construir otras estructuras del lenguaje, como generadores o comprehensions, y su uso es una de las piedras angulares dentro del paradigma de programación funcional en Python

## Generadores

Un *generator* se puede considerar como <a href="https://en.wikipedia.org/wiki/Syntactic_sugar">*azúcar sintáctico*</a> para la creación de objetos de tipo *iterator*. Es decir: <strong>un *iterator* escrito con una sintáxis más sencilla</strong>

Veámoslo con un ejemplo. Reescribamos el *iterator* del ejemplo anterior usando un *generator*. Concretamente una <strong>función generadora</strong>. Veremos lo que es esto a continuación.

In [92]:
# Iterador a partir de una función generadora
def fib():
    prev, curr = 0, 1
    while True:
        yield curr
        prev, curr = curr, prev + curr
        
f = fib()

# Recorremos nuestro iterador, llamando a next(). Dentro del for se llama automáticamente a iter(f)
print(0, end=' ')
for n in range(16):
    print(next(f), end=' ')

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 

Mucho más sencillo de escribir, ¿verdad? Además, ahora queda más clara la potencia de este tipo de estructura. Contiene un bucle infinito: *while True*. Y,a pesar de ello, no vamos a tener problemas de memoria.

Por otro lado, vuelve a aparecer la sentencia *yield*, que vimos fugazmente al estudiar los *context manager*. Ahora vamos a entenderla mejor.

Recordemos algo que ya se ha mencionado al explicar los iteradores, pero que es fundamental: cuando llamamos a *fib()*,<strong>no estamos ejecutando nada del código de nuestra función</strong>. 

Lo que sucede es que la función <strong>devuelve una estructura de tipo *generator*</strong> (de ahí que se denomine *generator function*). <strong>El cuerpo de la función en si, se va a ejecutar cuando se llame a la función *next()* con el generador como argumento</strong>. En ese momento, <strong>la función se ejecutará hasta que encuentre una llamada a *yield*</strong>, devolviéndose el valor que lance *yield* y <strong>dejando la función en *stand-by*</strong>.

<strong>La función seguirá *congelada* hasta que se vuelva a llamar a *next()*, momento en el cuál la función correrá hasta encontrar nuevamente una sentencia *yield*</strong>. Esto seguirá hasta que no haya más valores que *lanzar* mediante *yield*. En ese momento, diremos que el generador se ha *agotado*

<div class="alert alert-info">*yield* pausa una función generadora, mientras que *next()* hace que continúe en el punto donde se quedó</div>

¿Lo queremos aun más claro? Vayamos a un ejemplo más sencillo. Recuperemos el *iterable* a partir de una *comprehension list* del apartado de *Iterables*. Pero ahora, en lugar de generar una lista, vamos a generar un objeto de tipo *generator*.

La expresión con la que lo generamos es muy parecida a la *comprehension list*, pero usamos paréntesis *()* en lugar de corchetes *[]*. La expresión se llama *generation expression*. La veremos después.

In [96]:
# Generador a partir de una *generation expression*
numeros = (x*x for x in range(3))
for n in numeros:
    print(n, end=' ')

print()

# Si lo intentamos recorrer una segunda vez, esta vacio
for n in numeros:
    print(n, end=' ')
else:
    print("Generador vacío, tienes que volverlo a generar")

0 1 4 
Generador vacío, tienes que volverlo a generar


Para entenderlo mejor, vamos a escribir nosotros una *generator function* que genere el mismo objeto de tipo *generator* que la *generator expression*

In [98]:
# Creando nuestra propia *generator function*
def generatorFunction(limit):
    for n in range(limit):
        yield n*n # <--- Yield again!
        
numeros = generatorFunction(3)
for n in numeros:
    print(n, end=' ')

0 1 4 

*Deconstruyamos* nuestra función, para escribirla de otra forma...

In [97]:
# Deconstruyendo nuestra función generadora... 

def generatorFunction():
    # Ni siquiera necesitamos limite. Vamos a llamar a yield 3 veces, simulando lo que pasaría dentro del bucle
        yield 0
        yield 1
        yield 4
    
numeros = generatorFunction()

# En lugar de hacer un bucle, llamemos 3 veces seguidas a next(), que es lo que hace el for
print(next(numeros))
print(next(numeros))
print(next(numeros))

# Si intentamos acceder a un generador agotado, se lanza una excepción de tipo StopIteration
# La capturamos y terminamos silenciosamente, que es lo que hace for
try:
    print(next(numeros))
except StopIteration as e:
    pass # No hacemos nada


0
1
4


Para terminar este apartado, simplemente recopilemos algo que hemos ido viendo mientras explicábamos lo que es un *generator*. Hay <strong>dos tipos de *generator*</strong>

<ul>
    <li><strong>*generator function*</strong>: Una función que contiene la palabra *yield* en el cuerpo de la misma al menos una vez</li>
    <li><strong>*generator expression*</strong>: Una expresión de Python que genera un iterador</li>
</ul>

In [104]:
# Esto es una generator function. La podemos llamar hasta que se agote
def generatorFunction():
    # Ni siquiera necesitamos limite. Vamos a llamar a yield 3 veces, simulando lo que pasaría dentro del bucle
        yield 0
        yield 1
        yield 4

        
f = generatorFunction()
print(next(f))
print(next(f))
print(next(f))


# Esto es una *generator expression*. Como nos devuelve un iterador, podemos iterar sobre él hasta que se agote
lazy_squares = (x * x for x in [1, 2, 3, 4, 5, 6])

print(next(lazy_squares))
print(next(lazy_squares))
print(next(lazy_squares))
print(next(lazy_squares))
print(next(lazy_squares))

0
1
4
1
4
9
16
25


## Comprehensions

Las *comprehensions* son expresiones que transforman una secuencia en otra, y que permiten elegir condicionalmente qué elementos de la secuencia origen van a participar, y cómo, en la secuencia destino. Se consideran una alternativa a los bucles en algunas situaciones

<div class="alert alert-success"><strong>Buenas prácticas</strong>: Si puedes sustituir un bucle por una expresión de tipo *comprehension*, hazlo</div>

En la versión 2 de Python se introdujeron las *list comprehensions*, y en Python 3 se añadieron dos tipos más: las *dictionary comprehensions* y las *set comprehensions*. Las 3 son muy parecidas. Veámoslas

### List comprehensions

Las más antiguas y conocidas. Se construyen de la siguiente manera

In [106]:
# Creamos una lista a partir de otra mediante una list comprehension
a_list = [1, 9, 8, 4]
print([elem *2 for elem in a_list])

[2, 18, 16, 8]


Cualquier tipo de expresión Python es válida para generar los elementos de salida de una *list comprehension*

In [107]:
# Aquí aplicamos una llamada a función y un condicional
import math

a_list = [1, 9, 8, 4]
print([math.pi * elem for elem in a_list if elem % 2])

[3.141592653589793, 28.274333882308138]


### Dictionary comprehensions

La idea es la misma que en las list comprehensions, pero aplicado a diccionarios, Veamos un sencillo ejemplo, en el que cambiamos claves por valores

In [108]:
# Ejemplo sencillo: cambiar claves por valores
a_dict = {'a': 1, 'b': 2, 'c': 3}

# A destacar la llamada a .items() para obtener los elementos del diccionario
print({value:key for key, value in a_dict.items()})

{1: 'a', 2: 'b', 3: 'c'}


Es interesante ver qué sucede si intentamos hacer lo mismo pero teniendo como valor una lista, en lugar de un número

In [109]:
# Intentamos lo mismo, pero teniendo una lista como valor en lugar de un número
a_dict = {'a': [1, 2, 3], 'b': 4, 'c': 5}

print({value:key for key, value in a_dict.items()})

TypeError: unhashable type: 'list'

Lo que sucede es que una lista no puede ser utilizada como clave de un diccionario, porque una lista es lo que se conoce en Python como un *mutable object*:

<ul>
    <li><strong>mutable object</strong>: son objetos cuyo contenido puede cambiar sin que cambie el nombre asociado al mismo (la variable que los almacena). Ejemplos son listas, diccionarios y conjuntos (objetos tipo *set*). Estos objetos <strong>no pueden usarse como claves de diccionarios</strong></li>
    <li><strong>immutable object</strong>: son objetos cuyo contenido no puede cambiar sin que cambie el nombre asociado al mismo (la variable que los almacena). Ejemplos son números, cadenas (objetos tipo *string*) o tuplas. Estos objetos <strong>sí pueden usarse como claves de diccionarios</strong></li>
</ul>

Veamos la diferencia de manera clara en un ejemplo

In [113]:
# mutable vs immutable
s = "hola" # s es un string. Es immutable
l = ["h", "o", "l", "a"] # l es una lista. Es mutable.

# ok, se puede cambiar el objeto sin cambiar la variable que lo apunta
l[0] = "b"

# error, no se puede cambiar el objeto sin cambiar la variable que lo apunta. Tendríamos que cambiar el objeto entero.
s[0] = "b"

TypeError: 'str' object does not support item assignment

<div class="alert alert-warning">No intentes usar un objeto de tipo mutable como clave de un diccionario. Se producirá un *TypeError*</div>

¿Y cómo lo haríamos entonces para cambiar una simple letra de un string, o un elemento de una tupla?

Debemos convertir el objeto en lista, o cualquier otro elemento mutable, cambiarlo y volverlo a transformar en string. Ejemplo

In [ ]:
# Cambiando un objeto immutable
s = "hola"
l = list(s)
l[0] = "b"


### Set comprehensions

Son muy parecidos a los *dictionary comprehension*, pero los *set* solo tienen valores, en lugar de parejas clave:valor. 

Veamos un ejemplo

In [112]:
# set comprehension
a_set = set(range(10))

# El set no está ordenado
print({x**2 for x in a_set if x % 2})

{81, 1, 49, 9, 25}
